In [1]:
%pip install spacy --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m spacy download es_core_news_lg --quiet

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [7]:
!pip install faiss-cpu --quiet

In [23]:
import pandas as pd
import spacy
import numpy as np
import re
import os
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import faiss
import pprint

# Cargar corpus

In [9]:
file_path = r"C:\Users\karen\Documents\HumanidadesDigitales_git\BDD_Corpus\Corpus_completo_revisado.xlsx"
corpus_completo = pd.read_excel(file_path, engine='openpyxl')

In [10]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha'] = pd.to_datetime(corpus_completo['Fecha'], format='%d/%m/%Y', errors='coerce')

# Embeddings

In [14]:
# Generar embeddings de textos
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [15]:
textos = corpus_completo['Texto'].astype(str).tolist()  # Convertir a lista de strings

In [16]:
embeddings = modelo_embedding.encode(textos, convert_to_tensor=False).astype('float32')

In [19]:
np.save(r'Out/embeddingsIR.npy', embeddings)

In [ ]:
embeddings = np.load(r'Out/embeddingsIR.npy')

In [20]:
# Crear índice FAISS
indice = faiss.IndexFlatL2(embeddings.shape[1])
indice.add(embeddings)

# Consulta

In [24]:
# Buscar texto similar a una consulta
consulta = ["¿Cómo evolucionó la percepción sobre el proceso de paz entre enero de 2019 y diciembre de 2020?"]

embedding_consulta = modelo_embedding.encode(consulta, convert_to_tensor=False).astype('float32')
distancias, indices = indice.search(embedding_consulta, k=20)

In [26]:
#Resultados
print("Textos más similares:")
for idx in indices[0]:
    print(f"Índice: {idx}")
    pprint.pp("Texto: " + textos[idx])
    pprint.pp("---------------------------------------")

Textos más similares:
Índice: 10234
('Texto: El título del proceso de paz ya es obsoleto. No tenemos paz, así que '
 'ni hablar de que sea estable ni duradera. Es posible que la implementación '
 'del Acuerdo eventualmente sirva para apaciguar al Eln, las disidencias de '
 'las Farc y los diversos grupos armados que operan sin un discurso político '
 'que les dé una pátina de legitimidad, o que operan con discursos de derecha. '
 'Lo más probable es que no.\n'
 '\n'
 '¿Cuál fue entonces el valor de la firma de un acuerdo que, si bien promete '
 'paz, resulta que antecedió el más alto nivel de asesinatos de líderes '
 'comunitarios, activistas y excombatientes de los últimos 20 años?\n'
 '\n'
 'Hay tres respuestas: una contrafactual, otra efectiva y otra prospectiva.\n'
 '\n'
 'La contrafactual es que el país estaría en una situación menos crítica si el '
 'Acuerdo de Paz se hubiese implementado desde el principio, con celeridad, '
 'comenzando por las zonas más complejas y que hoy son 